How do atomic operations translate from CUDA `C` to `SASS` ?
=======================================================

This example demonstrates how `ReProspect` can be used to quickly analyse how a given `C` operation is translated into its corresponding `SASS` instructions.

This approach offers several advantages:
- **Pedagogical** — useful for teaching and learning
- **Comparative** — helps understand how the mapping from `C` to `SASS` evolves across architectures

Source code
-----------

Let's take a closer look at `atomicAdd`, described [here](https://docs.nvidia.com/cuda/cuda-c-programming-guide/#atomicadd).
The kernel atomically adds a source array to a destination array, element-wise.

In [ ]:
CODE = """\
#include "cuda.h"

__global__ void my_kernel(int * __restrict__ const dst, const int* __restrict__ const src) {
    const auto index = blockIdx.x * blockDim.x + threadIdx.x;
    atomicAdd(&dst[index], src[index]);
}
"""

Compilation
-----------

Compile the source code for a few architectures of your choice.
As this analysis won't actually run the executable, any architecture supported by ``nvcc`` should work.

In [ ]:
import logging
import pathlib
import subprocess
import tempfile

from reprospect.tools import architecture

logging.basicConfig(level = logging.INFO)

ARCHES = [
    architecture.NVIDIAArch.from_compute_capability(80),
    architecture.NVIDIAArch.from_compute_capability(90),
    architecture.NVIDIAArch.from_compute_capability(120),
]

logging.info(subprocess.check_output(('nvcc', '--version')).decode())

with tempfile.TemporaryDirectory(delete = False) as tmpdir:
    for arch in ARCHES:
        source = pathlib.Path(tmpdir) / f'atomic.{arch.as_sm}.cu'
        output = pathlib.Path(tmpdir) / f'atomic.{arch.as_sm}'

        source.write_text(CODE)

        subprocess.check_call(('nvcc', f'--generate-code=arch={arch.as_compute},code=[{arch.as_sm}]', '-O3', '-c', source, '-o', output))

Binary analysis
---------------

Let's inspect the generate `SASS` instructions.

In [ ]:
from reprospect.tools import binaries

cuobjdump = {}

for arch in ARCHES:
    cuobjdump[arch], _ = binaries.CuObjDump.extract(
        file = pathlib.Path(tmpdir) / f'atomic.{arch.as_sm}',
        arch = arch,
        cwd = pathlib.Path(tmpdir),
        cubin = f'atomic.1.{arch.as_sm}.cubin',
    )

In [ ]:
from reprospect.tools.sass import Decoder

SIGNATURE = 'my_kernel(int *, const int *)'

for arch in ARCHES:
    logging.info(Decoder(code = cuobjdump[arch].functions[SIGNATURE].code))

For the kernel defined above, the atomic operation is always translated into a *Reduction Operation on Generic Memory* on the target architectures.

However, the exact encoding varies with compute capability:
- On architectures **prior to** ``Hopper``, it appears as ``RED``.
- On architectures **starting from** ``Hopper``, it appears as ``REDG``.

In [ ]:
for arch, dump in cuobjdump.items():
    if arch.compute_capability < 90:
        assert 'RED.E.ADD.STRONG.GPU' in dump.functions[SIGNATURE].code
    else:
        assert 'REDG.E.ADD.STRONG.GPU' in dump.functions[SIGNATURE].code